# Combining satellite data with tidal modelling <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the `DEA Sandbox` environment only
* **Products used:** 
[ga_ls8c_ard_3](https://explorer.dea.ga.gov.au/ga_ls8c_ard_3)

## Background

Ocean tides are the periodic rise and fall of the ocean caused by the gravitational pull of the moon and sun and the earth's rotation. 
Tides in coastal areas can greatly influence how these environments appear in satellite imagery as water levels vary by up to 12 metres (e.g. in north-western Australia).
To be able to study coastal environments and processes along Australia's coastline, it is vital to obtain data on tidal conditions at the exact moment each satellite image was acquired.

## Description
This notebook demonstrates how to combine remotely sensed imagery with information about ocean tides using functions from the [`dea_tools.coastal` module](../Tools/dea_tools/coastal.py), allowing us to analyse satellite imagery by tidal stage (e.g. low, high, ebb, flow). 
These functions use the [Finite Element Solution 2014 (FES2014) tidal model](https://os.copernicus.org/articles/17/615/2021/) to calculate the height (relative to mean sea level, i.e. approximately equivalent to the Australian Height Datum or AHD) and stage of the tide at the exact moment each satellite image was acquired. 

These tide modelling tools and tide models underpin DEA products including [DEA Coastlines](../DEA_products/DEA_Coastlines.ipynb) and [DEA Intertidal](../DEA_products/DEA_Intertidal.ipynb).

The notebook demonstrates how to:

1. Model tide heights for specific coordinates and times using the `model_tides` function
2. Model tide heights for each satellite observation using the `tidal_tag` function
3. Use tide height data to produce median composites of the coast at low and high tide
4. Swap a dataset's dimensions to make it easier to select imagery from low to high tide.
5. Compute ebb or flow tide phase data to determine whether water levels were rising or falling in each satellite observation

Advanced tools:

6. Spatially model tides into each pixel of a satellite dataset using `pixel_tides`
7. Evaluate potential biases in the tides observed by a satellite using `tidal_stats`

***

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages


In [ ]:
pip install git+https://github.com/tsutterley/pyTMD.git

In [ ]:
pip install git+https://github.com/tsutterley/pyTMD.git

In [ ]:
pip install pyTMD==2.1.3

In [1]:
%load_ext autoreload
%autoreload 2

import datacube
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datacube.utils.masking import mask_invalid_data

import sys

sys.path.insert(1, "../Tools/")
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.coastal import model_tides, tidal_tag, pixel_tides, tidal_stats

import os



In [2]:
# %%timeit -n 1 -r 1


from pyTMD.compute import tide_elevations
import pandas as pd

out = tide_elevations(
    x=[122.14],
    y=[-17.9],
    delta_time=pd.date_range("2020", "2021", periods=2),
    DIRECTORY="/gdata1/data/tide_models_clipped/",
    MODEL="HAMTIDE11",
    EPSG=4326,
    TIME="datetime",
    EXTRAPOLATE=True,
)

out

# from pyTMD.compute_tide_corrections import compute_tide_corrections

# out = compute_tide_corrections(
#     x=[122.14],
#     y=[-17.9],
#     delta_time=pd.date_range("2020", "2021", periods=2),
#     DIRECTORY="/gdata1/data/tide_models_clipped/",
#     MODEL="HAMTIDE11",
#     EPSG=4326,
#     TIME="datetime",
#     EXTRAPOLATE=True,
# )

# out


masked_array(data=[-2.6231363277730178, -1.8025663625134685],
             mask=[False, False],
       fill_value=nan)

In [ ]:
out.data

In [ ]:
import numpy as np

In [ ]:
# %%timeit -n 1 -r 1

# Set of times to model tides for
date_list = pd.date_range(start="2017-01-01", end="2019-12-31", freq="3h")

y, x = -38.337069,144.744488
y = np.linspace(-38, -39, 10)
x = np.linspace(144, 145, 10)

os.environ["DEA_TOOLS_TIDE_MODELS"] = "/gdata1/data/tide_models_clipped"

# Run the FES2014 tidal model
tide_df = model_tides(
    x=x,
    y=y,
    model=["FES2022"],
    output_format="wide",
    method="bilinear",
    time=date_list,
)

In [ ]:
# %%timeit -n 1 -r 1

# Set of times to model tides for
date_list = pd.date_range(start="2017-01-01", end="2019-12-31", freq="3h")

y, x = -38.337069,144.744488
y = np.linspace(-38, -39, 1000)
x = np.linspace(144, 145, 1000)

os.environ["DEA_TOOLS_TIDE_MODELS"] = "/gdata1/data/tide_models_clipped"

# Run the FES2014 tidal model
tide_df = model_tides(
    x=x,
    y=y,
    model=[
    "FES2022",
    "FES2014",
    "TPXO9-atlas-v5",
    "EOT20",
    "HAMTIDE11",
    "GOT4.10",
    "FES2012",
    "TPXO8-atlas-v1",
],
    output_format="wide",
    method="spline",
    time=date_list,
    # parallel_splits=1,
    # crop=True,
)

In [ ]:
# %%timeit -n 1 -r 1

# Set of times to model tides for
date_list = pd.date_range(start="2017-01-01", end="2019-12-31", freq="3h")

y, x = -38.337069,144.744488
y = np.linspace(-38, -39, 1000)
x = np.linspace(144, 145, 1000)

os.environ["DEA_TOOLS_TIDE_MODELS"] = "/gdata1/data/tide_models_clipped"

# Run the FES2014 tidal model
tide_df_crop = model_tides(
    x=x,
    y=y,
    model=[
    "FES2022",
    "FES2014",
    "TPXO9-atlas-v5",
    "EOT20",
    "HAMTIDE11",
    "GOT4.10",
    "FES2012",
    "TPXO8-atlas-v1",
],
    output_format="wide",
    method="spline",
    time=date_list,
    # parallel_splits=1,
    crop=True,
)

In [ ]:
# %%timeit -n 1 -r 1

# ymax, xmin = -18.10, 122.10
# ymin, xmax = -18.30, 122.30

# out = tide_elevations(
#     x=np.array([122.21, 122.20]),
#     y=np.array([-18.20, -18.20]),
#     delta_time=pd.date_range(start="2022-09-01", end="2022-09-30", freq="1h"),
#     DIRECTORY="/gdata1/data/tide_models_clipped/",
#     MODEL="FES2022",
#     EPSG=4326,
#     TIME="datetime",
#     METHOD="bilinear",
# )

# out

In [ ]:
# %%timeit -n 1 -r 1

# ymax, xmin = -18.10, 122.10
# ymin, xmax = -18.30, 122.30

# out = tide_elevations(
#     x=np.array([122.21, 122.20]),
#     y=np.array([-18.20, -18.20]),
#     delta_time=pd.date_range(start="2022-09-01", end="2022-09-30", freq="1h"),
#     DIRECTORY="/gdata1/data/tide_models_clipped/",
#     MODEL="FES2022",
#     EPSG=4326,
#     TIME="datetime",
#     METHOD="spline",
# )

# out

In [ ]:
# from pyTMD.io import model, extract_constants

In [ ]:
# pytmd_model = model(
#             "/gdata1/data/tide_models_test/", format="netcdf", compressed=False
#         ).elevation("FES2022")

In [ ]:
# %%timeit -n 1 -r 1

# import pyTMD.io

# ymax, xmin = -18.10, 122.10
# ymin, xmax = -18.30, 122.30

# out = pyTMD.io.FES.extract_constants(
#             122.21,
#             -18.20,
#             pytmd_model.model_file,
#             type=pytmd_model.type,
#             version=pytmd_model.version,
#             # crop=True,
#             # bounds=[xmin, xmax, ymin, ymax],
#             method="bilinear",
#             extrapolate=True,
#             # cutoff=cutoff,
#             scale=pytmd_model.scale,
#             compressed=pytmd_model.compressed,
#         )

### Connect to the datacube


In [ ]:
dc = datacube.Datacube(app="Tidal_modelling")

## Modelling tide heights using `model_tides`
To simply model tide heights for a specific location and set of times, we can use the `dea_tools.coastal.model_tides` function. 
For example, we can model hourly tides across a one month period (September 2022):

In [ ]:
# Set of times to model tides for
date_list = pd.date_range(start="2017-01-01", end="2019-12-31", freq="3h")

y, x = -38.337069,144.744488

os.environ["DEA_TOOLS_TIDE_MODELS"] = "/gdata1/data/tide_models_clipped"

# Run the FES2014 tidal model
tide_df = model_tides(
    x=[x],
    y=[y],
    model=[
    "FES2022",
    "FES2014",
    "TPXO9-atlas-v5",
    "EOT20",
    "HAMTIDE11",
    "GOT4.10",
    "FES2012",
    "TPXO8-atlas-v1",
],
    output_format="wide",
    method="bilinear",
    time=date_list,
)

# os.environ["DEA_TOOLS_TIDE_MODELS"] = "/gdata1/data/tide_models_test"

tide_df_extrap = model_tides(
    x=[x],
    y=[y],
    model=[
    "FES2022",
    # "FES2014",
    # "TPXO9-atlas-v5",
    # "EOT20",
    # "HAMTIDE11",
    # "GOT4.10",
    # "FES2012",
    # "TPXO8-atlas-v1",
],
    output_format="wide",
    time=date_list,
    method="bilinear",
    extrapolate=True,
)

# Add tide to dataframe
tide_df["FES2022_extrap"] = tide_df_extrap.FES2022

# Print outputs
tide_df.head()

In [ ]:
((tide_df.reset_index(["x", "y"], drop=True)).resample("1M").max() * 2).plot(figsize=(10, 10));

In [ ]:
# !cp -r /gdata1/data/tide_models/fes2022b/ocean_tide_extrapolated /gdata1/data/tide_models_test/fes2022b/ocean_tide_extrapolated

Tide heights for each time and coordinate are included in the `tide_m` column above  (representing tide height in metres relative to Mean Sea Level).

We can also plot out resulting tides to view how tides changed across this month. 
By looking at the y-axis, we can see that tides ranged from a minimum of ~-4 metres up to a maximum of +4 metres relative to Mean Sea Level:

In [ ]:
ax = (tide_df.reset_index(["x", "y"], drop=True).resample("1M").max() * 2).rename({"tide_m": "no extrapolat"}, axis=1).plot(figsize=(10, 10));
(tide_df_extrap.reset_index(["x", "y"], drop=True).resample("1M").max() * 2).rename({"tide_m": "extrapolated"}, axis=1).plot(ax=ax)

In [ ]:
(tide_df_extrap.reset_index(["x", "y"], drop=True).resample("1M").max() * 2).plot(figsize=(10, 10));

In [ ]:
tide_df_extrap

### "One-to-many" and "one-to-one" modes

By default, the `model_tides` function operates in **"one-to-many"** mode, which will model tides for every requested timestep, at every requested location.
For example, if we provided five x, y coordinates and five timesteps, the function would return:
```
5 locations * 5 timesteps = 25 modelled tides
```

However, often you may have a list of locations and matching timesteps.
Using **"one-to-one"** mode, we can model tides for only these exact pairs of locations and times:
```
5 timesteps at 5 locations = 5 modelled tides
```

To demonstrate "one-to-one" mode, imagine we have a `pandas.Dataframe` where each row contains unique site locations and times:

In [ ]:
sites_df = pd.DataFrame({
    "time": pd.date_range(start="2022-09-01", end="2022-09-30", periods=5),
    "x": [122.21, 122.22, 122.23, 122.24, 122.25],
    "y": [-18.20, -18.20, -18.20, -18.20, -18.20]
})

sites_df

Using "one-to-one" mode, we can model a tide height for each row in our dataframe, and add it as a new dataframe column:

In [ ]:
# Model tides in "one-to-one" mode
tide_df = model_tides(
    x=sites_df.x,
    y=sites_df.y,
    time=sites_df.time,
    mode="one-to-one",
)

# Add results as a new datframe column
sites_df["tide_height"] = tide_df.tide_m.values
sites_df.style.set_properties(**{"background-color": "#FFFF8F"}, subset=["tide_height"])

## Modelling tide heights for each satellite observation using `tidal_tag`

However, often it is valuable to estimate the height of the tide at the exact moment satellite images were taken over a coastline.
This can help sort and filter images by tide height, allowing us to learn more about how coastal environments respond to the effect of changing tides.

To demonstrate how this can be done, we first need to load in an example time series of satellite imagery.

### Set up data query

First we set up a query to define the area, time period and other parameters required for loading data.
In this example, we will load several years of Landsat 8 data for intertidal mud flats **south of Broome in Western Australia**.
We load the `'nbart_red', 'nbart_green', 'nbart_blue'` bands so that we can plot the data as true colour imagery.

> The `dask_chunks` parameter allows us to [use Dask](https://examples.dask.org/xarray.html) to lazily load data rather than load data directly into memory, which can take a long time and large amounts of memory.
Lazy loading can be a very useful approach for when you need to load large amounts of data without crashing your analysis.
In coastal applications, it allows us to load (using either `.compute()` or by plotting our data) only a small subset of observations from our entire time series (e.g. only low or high tide observations) without having to load the entire dataset into memory first, which can greatly decrease processing times.
>
> For more information about using Dask, refer to the [Parallel processing with Dask](../Beginners_guide/09_Parallel_processing_with_Dask.ipynb) notebook.

In [ ]:
# Set up data load query
query = {
    "x": (122.15, 122.26),
    "y": (-18.14, -18.24),
    "time": ("2020-01", "2020-06"),
    "measurements": ["nbart_red", "nbart_green", "nbart_blue"],
    "group_by": "solar_day",
    "dask_chunks": {},
}

We can preview the area that we will load data for:

In [ ]:
display_map(x=query['x'], y=query['y'])

### Load satellite time-series

To obtain some satellite data to analyse, we use the `load_ard` function to import a time series of Landsat 8 observations as an `xarray.Dataset`.
The input data does not need to be from Landsat: any remotely-sensed imagery with timestamps and spatial coordinates provide enough data to run the tidal model. 

In [ ]:
# Load available data from Landsat 8
ds = load_ard(
    dc=dc,
    products=["ga_ls8c_ard_3"],
    mask_pixel_quality=False,
    cloud_cover=(0, 10),  # load only scenes with <10% cloud
    **query
)

# Set nodata to NaN and print output
ds = mask_invalid_data(ds)
ds

### "Tagging" each satellite image with tide height metadata

We can now use the `tidal_tag` function from `dea_tools.coastal` to "tag" or associate each satellite observation in our time series with a tide height relative to mean sea level (i.e. approximately equivalent to the Australian Height Datum or AHD).
This function uses the time and date of acquisition and the geographic location of each satellite observation as inputs to the [Finite Element Solution 2014 (FES2014) tidal model](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/description-fes2014.html):

> **FES2014** is the last version of the FES (Finite Element Solution) tide model developed in 2014-2016. It is an improved version of the FES2012 model. This new FES2014 model has been developed, implemented and validated by the LEGOS, NOVELTIS and CLS, within a CNES funded project. FES2014 takes advantage of longer altimeter time series and better altimeter standards, improved modelling and data assimilation techniques, a more accurate ocean bathymetry and a refined mesh in most of the shallow water regions. Special efforts have been dedicated to address the major non-linear tides issue and to the determination of accurate tidal currents.

The function will automatically select a tide modelling location based on the dataset centroid.
It will then output modelled tide heights as a new `tide_m` variable in the `xarray.Dataset` (the variable should appear under `Data variables` below):

In [ ]:
# Model tide heights
ds_tidal = tidal_tag(ds)

# Print output data
ds_tidal

We can easily plot this new variable to inspect the range of tide heights observed by the satellites in our time series.
In this example, our observed tide heights range from approximately -2.0 to 4.0 m relative to Mean Sea Level:

In [ ]:
ds_tidal.tide_m.plot();

### Example tide height analysis
To demonstrate how tidally tagged images can be used to produce composites of high and low tide 
imagery, we can compute the lowest 20% and highest 20% of tide heights, and use these to filter our observations. 
We can then combine and plot these filtered observations to visualise how the landscape appears at low and high tide:

In [ ]:
# Calculate the lowest and highest 20% of tides
lowest_20, highest_20 = ds_tidal.tide_m.quantile([0.2, 0.8]).values

# Filter our data to low and high tide observations
filtered_low = ds_tidal.where(ds_tidal.tide_m <= lowest_20, drop=True)
filtered_high = ds_tidal.where(ds_tidal.tide_m >= highest_20, drop=True)

# Take the simple median of each set of low and high tide observations to
# produce a composite (alternatively, observations could be combined
# using a geomedian to keep band relationships consistent)
median_low = filtered_low.median(dim="time", keep_attrs=True)
median_high = filtered_high.median(dim="time", keep_attrs=True)

# Combine low and high tide medians into a single dataset and give
# each layer a meaningful name
ds_highlow = xr.concat([median_low, median_high], dim="tide_m")
ds_highlow["tide_m"] = ["Low tide", "High tide"]

# Plot low and high tide medians side-by-side
rgb(ds_highlow, col="tide_m")

### Swapping dimensions

The `tidal_tag` function allows you to use tide heights as the primary dimension in the dataset, rather than time.
Setting `swap_dims=True` will swap the `time` dimension in the original `xarray.Dataset` to the new `tide_m` variable.

In [ ]:
# Model tide heights
ds_tidal = tidal_tag(ds, swap_dims=True)

# Print output data
ds_tidal

The dataset now contains three dimensions: `tide_m`, `x` and `y`. 
This can make it easier to analyse the data with respect to tide, e.g. plotting the lowest and highest tide image in our dataset:

In [ ]:
# Plot first (lowest tide) and last (highest tide) images
rgb(ds_tidal, index_dim="tide_m", index=[0, -1])

> **Note:** The white dots in the images above are false positives in the automated cloud mask caused by bright sandy shorelines being mistaken for clouds.

### Modelling ebb and flow tidal phases

The `tidal_tag` function also allows us to determine whether each satellite observation was taken while the tide was rising/incoming (flow tide) or falling/outgoing (ebb tide) by setting `ebb_flow=True`.
This is achieved by comparing tide heights 15 minutes before and after the observed satellite observation.

Ebb and flow data can provide valuable contextual information for interpreting satellite imagery, particularly in tidal flat or mangrove forest environments where water may remain in the landscape for considerable time after the tidal peak.

In [ ]:
# Model tide heights
ds_tidal = tidal_tag(ds, ebb_flow=True)

# Print output data
ds_tidal

We now have data giving us the both the tide height and tidal phase ('ebb' or 'flow') for every satellite image:

In [ ]:
ds_tidal[["time", "tide_m", "ebb_flow"]].to_dataframe().drop("spatial_ref", axis=1)

We could for example use this data to filter our observations to keep ebbing phase observations only:

In [ ]:
ds_tidal.where(ds_tidal.ebb_flow == "Ebb", drop=True)

## Advanced

### Modelling tides for each pixel in a satellite time series using `pixel_tides`

The previous examples show how to model a single tide height for each satellite image using the centroid of the image as a tide modelling location. 
However, in reality tides vary spatially, potentially up to several metres across a distance of kilometres in areas of complex tidal dynamics. 
This means that an individual satellite image can contain a range of tide height conditions.

To capture this spatial variability in tide heights, we can use the `pixel_tides` function from `dea_tools.coastal`.
For efficient processing, this function first models tides into a low resolution 5000 m grid surrounding each satellite image in our time series.
This lower resolution data will also include a buffer around the extent of our data tides can be modelled seamlessly across analysis boundaries:

In [ ]:
# Extract a subset of our satellite data above to make plotting easier
ds_subset = ds.isel(time=slice(0, 3))

# Model tides spatially using `pixel_tides`
tides_lowres = pixel_tides(ds_subset, resample=False)

If we plot the resulting data, we can see that we now have 2D tide surfaces for each timestep in our data (instead of the single tide height per timestamp returned by the `tidal_tag` function).

Blue values below indicate low tide pixels, while red indicates high tide pixels.
If you look closely at the second and third timestep below, you may also be able to see some spatial variability in tide heights within each timestep, with slight variations in tide heights along the left (west) side of the study area:

In [ ]:
tides_lowres.plot.imshow(col="time", vmin=-1, vmax=1, cmap='RdBu')

#### Reprojecting tide heights back into original high resolution spatial grid
By setting `resample=True`, we can use interpolation to re-project our low resolution tide data back into the resolution of our satellite image, resulting in an individual tide height value for every single pixel in our dataset through time and space:

In [ ]:
# Model tides spatially using `pixel_tides`
tides_highres, tides_lowres = pixel_tides(ds_subset, resample=True)

tides_highres.plot.imshow(col="time", vmin=-1, vmax=1, cmap="RdBu")

`tides_highres` will have exactly the same dimensions as `ds_subset`, with a unique tide height for every satellite pixel:

In [ ]:
ds_subset.sizes

In [ ]:
tides_highres.sizes

Because of this, our stack of tides can be added as an additional 3D variable in our dataset:

In [ ]:
ds_subset["tide_m"] = tides_highres
ds_subset

#### Calculating min/max/median/quantiles of tide heights for each pixel

Min, max or any specific quantile of all tide heights observed over a region can be calculated for each pixel by passing in a list of quantiles/percentiles. 

This calculation is performed on the low resolution modelled tide data before reprojecting to higher resolution, so should be faster than calculating min/max/median tide at high resolution:

In [ ]:
tides_highres_quantiles, tides_lowres_quantiles = pixel_tides(
    ds_subset, resample=True, calculate_quantiles=(0, 0.5, 1)
)

In [ ]:
tides_highres_quantiles.plot.imshow(col="quantile")

#### Pixel-based tides for custom times
Instead of using times contained in the `time` dimension of our dataset, we can also calculate pixel-based tides for a custom set of times:

In [ ]:
custom_times = pd.date_range(start="2022-01-01", end="2022-01-02", freq="6H")

tides_highres, tides_lowres = pixel_tides(ds_subset, resample=True, times=custom_times)

In [ ]:
tides_highres.plot.imshow(col="time")

### Evaluating tidal biases using `tidal_stats`

The complex temporal behaviour of tides mean that a sun synchronous sensor like Landsat [does not observe the full range of the tidal cycle at all locations](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3).
Biases in the proportion of the tidal range observed by satellites can prevent us from obtaining data on areas of the coastline exposed or inundated at the extremes of the
tidal range. 
This can risk gaining misleading insights into the true extent of the area of the coastline affected by tides, and make it difficult to compare high or low tide images fairly in different locations.

The `tidal_stats` function can assist in evaluating how the range of tides observed by satellites compare to the full tidal range.
It works by using our tidal model to to model all available tide heights at a regular interval (every two hours by default) across the entire time period covered by the input satellite time series dataset.
This is then compared against the tide heights in observed by the satellite and used to calculate a range of statistics and a plot that summarises potential biases in the data.

> For a more detailed discussion of the issue of tidal bias in sun-synchronous satellite observations of the coastline, refer to the ['Limitations and future work' section in Bishop-Taylor et al. 2018](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3).


In [ ]:
out_stats = tidal_stats(ds)

The function also outputs a `pandas.Series` object containing a set of statistics that compare the observed vs. full modelled tidal ranges.
These statistics include: 

* **tidepost_lat:** latitude used for modelling tide heights
* **tidepost_lon:** longitude used for modelling tide heights
* **observed_min_m:** minimum tide height observed by the satellite (in metre units)
* **all_min_m:** minimum tide height from full modelled tidal range (in metre units)
* **observed_max_m:** maximum tide height observed by the satellite (in metre units)
* **all_max_m:** maximum tide height from full modelled tidal range (in metre units)
* **observed_range_m:** tidal range observed by the satellite (in metre units)
* **all_range_m:** full modelled tidal range (in metre units)
* **spread_m:** proportion of the full modelled tidal range observed by the satellite (see [Bishop-Taylor et al. 2018](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3))
* **low_tide_offset:** proportion of the lowest tides never observed by the satellite (see [Bishop-Taylor et al. 2018](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3))
* **high_tide_offset:** proportion of the highest tides never observed by the satellite (see [Bishop-Taylor et al. 2018](https://www.sciencedirect.com/science/article/pii/S0272771418308783#sec3))

In [ ]:
out_stats

---

## Additional information

**FES2014:** Tidal modelling is provided by the FES2014 global tidal model, implemented using functions from the [pyTMD Python package](https://pytmd.readthedocs.io/en/latest/getting_started/Citations.html). FES2014 was produced by NOVELTIS, LEGOS, CLS Space Oceanography Division and CNES. It is distributed by AVISO, with support from CNES (https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/description-fes2014.html).

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [GitHub](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** July 2024

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
<!-- Browse all available tags on the DEA User Guide's [Tags Index](https://knowledge.dea.ga.gov.au/genindex/) -->